In [1]:
conda info --envs

# conda environments:
#
base                     /anaconda
azureml_py310_sdkv2      /anaconda/envs/azureml_py310_sdkv2
azureml_py38             /anaconda/envs/azureml_py38
azureml_py38_PT_TF       /anaconda/envs/azureml_py38_PT_TF
inference_env            /anaconda/envs/inference_env


Note: you may need to restart the kernel to use updated packages.


In [5]:
!source activate inference_env

In [1]:
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset


In [2]:
ws = Workspace.from_config()

In [3]:
ws.datasets

{'titatnic_dataset': DatasetRegistration(id='94c42d38-6d85-49b1-814c-c51b5ffac997', name='titatnic_dataset', version=1, description='', tags={})}

In [9]:
ws.models

{'titanic_model': Model(workspace=Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1'), name=titanic_model, id=titanic_model:9, version=9, tags={'data': 'titanic', 'model': 'classification'}, properties={})}

In [10]:
import joblib


In [4]:
#m1 = joblib.load(ws.models['titanic_model'])

In [5]:
## Use the below to prevent the 'No usable environment found' error

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants

myenv = Environment.from_conda_specification(name="myenv", file_path="scoringConfig.yml")
myenv

No Python version provided, defaulting to "3.8.13"


{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
           

In [13]:
#model = ws.models['titanic_model']
from azureml.core.model import Model
model = Model(ws, name="titanic_model", version=9)
model

Model(workspace=Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1'), name=titanic_model, id=titanic_model:9, version=9, tags={'data': 'titanic', 'model': 'classification'}, properties={})

In [7]:
 ws.models

{'titanic_model': Model(workspace=Workspace.create(name='titanic-aml-v1', subscription_id='ffe8e859-0cfb-4c23-8708-291600dadedf', resource_group='titanic-rg-v1'), name=titanic_model, id=titanic_model:9, version=9, tags={'data': 'titanic', 'model': 'classification'}, properties={})}

In [19]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'Titanic classification service')

aci_service_name = 'titanic-all1'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

titanic-all1


/tmp/ipykernel_90311/373667320.py:15: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-19 12:17:55+00:00 Creating Container Registry if not exists.
2022-12-19 12:17:55+00:00 Registering the environment.
2022-12-19 12:17:56+00:00 Use the existing image.
2022-12-19 12:17:56+00:00 Generating deployment configuration.
2022-12-19 12:17:57+00:00 Submitting deployment to compute.
2022-12-19 12:18:00+00:00 Checking the status of deployment titanic-all1..
2022-12-19 12:19:42+00:00 Checking the status of inference endpoint titanic-all1.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: ab95e457-0955-47fe-b6bb-c152b9ba96b5
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: titanic-all1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image titanicv1amlcr.azurecr.io/azureml/azureml_82455aa012066c116bfeb275ff99e1ba locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more informat

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: ab95e457-0955-47fe-b6bb-c152b9ba96b5
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: titanic-all1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image titanicv1amlcr.azurecr.io/azureml/azureml_82455aa012066c116bfeb275ff99e1ba locally. Please refer to https://aka.

In [16]:
#pip install azure-ai-ml

In [17]:
os.getenv("AZUREML_MODEL_DIR")